# [Module 0.0] Option: 아마존 상품 리뷰 데이터 준비

**다운로드 받은 Repo에 'preproc_data/translated/full_translated_grocery_22009.csv'
파일이 존재하면 이 노트북은 스킵하셔도 됩니다.**

적당한 한국어로 상품 리뷰의 데이터를 구하기가 어려워서, Amazon.com의 상품 리뷰를 한글로 번역하여 데이터를 준비 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 아마존 리뷰데이타 중에서 Grocery 항목의 파일 1개를 다운로드 합니다.
- 리뷰에서 등급 1-2 만을 추출합니다. 1-2를 부정 리뷰로 간주 합니다.
    - 22009 의 부정 리뷰를 사용 합니다.
- 영어를 한국어로 번역 합니다.
- Amazon Translate에 번역을 요청 합니다. (병렬로 4개의 작업이 동시에 번역 요청 합니다.)

In [1]:
import pandas as pd

## 아마존 리뷰 데이터 셋트 다운로드

아래에서 grocery 의 데이터 파일 하나만을 사용했습니다. 다른 카테고리를 사용하시고 싶다면 prefix1, prefix2 주석을 제거하고 사용하세요

In [2]:
import os, sagemaker

def download_amazon_review(basic_folder = 's3://amazon-reviews-pds/tsv', prefix='', down_folder=''):
    s3_source_path = os.path.join(basic_folder, prefix)
    sagemaker.s3.S3Downloader.download(s3_source_path, down_folder)
    print(f'{s3_source_path} is downloaded')


amazon_data = 'data'
os.makedirs(amazon_data, exist_ok=True)

# prefix1 = 'amazon_reviews_us_Apparel_v1_00.tsv.gz'
# download_amazon_review(prefix = prefix1, down_folder = amazon_data)
# prefix2 = 'amazon_reviews_us_Beauty_v1_00.tsv.gz'
# download_amazon_review(prefix = prefix2, down_folder = amazon_data)
prefix3 = 'amazon_reviews_us_Grocery_v1_00.tsv.gz'
download_amazon_review(prefix = prefix3, down_folder = amazon_data)

s3://amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz is downloaded


## 리뷰에서 부정적인 것만 추출
점수 등급이 1-5에서 1-2만 추출 합니다. 즉 1-2 를 부정적이라고 간주 합니다.

In [3]:
import csv

def get_review_data(data_folder, file_path, nrows=10000):
    '''
    rating이 1,2 인 것만 제공
    '''
    file_path = os.path.join(amazon_data, file_path)
    df = pd.read_csv(file_path, 
                     delimiter='\t', 
                     nrows= nrows,
                     quoting=csv.QUOTE_NONE,
                     compression='gzip')
    df = df[df.star_rating.isin([1,2])]
    print("data set shape: ", df.shape)
    cols = ['review_id','review_body','product_category','star_rating','review_date' ]
    df = df[cols]

    return df

grocery_df = get_review_data(data_folder = amazon_data, file_path = prefix3, nrows=200000)

data set shape:  (22009, 15)


In [4]:
grocery_df.head()

,review_id,review_body,product_category,star_rating,review_date
5,RIG9AWFOGRDVO,The popcorn was stale.,Grocery,2,2015-08-31
9,R1ODXB3C9UP3NL,Used to be a decent product. Disgusting now a...,Grocery,1,2015-08-31
17,R3T6TTD2IN0EFZ,I bought this from a local super market on a w...,Grocery,1,2015-08-31
23,R9MISLBRG08FX,not craze about these. nothing really wrong wi...,Grocery,1,2015-08-31
40,R1DIYK2UJ1C7FC,This product is beautifully labelled. That's a...,Grocery,1,2015-08-31


## 영어를 한국어로 번역

#### Util 함수

In [5]:
import re
import numpy as np
from multiprocessing import Pool

def process_df_parallel(df, func, n_cores=4):
    '''
    dataframe의 병렬 수행
    '''
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

import pandas as pd
import boto3

translate = boto3.client(service_name='translate', region_name='ap-northeast-2', use_ssl=True)

def translate_en_df(df, source_lang='en', target_lang = 'ko', max_length=1000):
    '''
    review_body를 한국어로 번역함
    내부에서 translate_kr 을 호출함
    '''

    df.insert(1, 'review_body_kr', df.review_body.apply(lambda x: translate_kr(x, source_lang,target_lang, max_length)))
    # print("review_body translation is done")
        
    return df

import pandas as pd
def translate_kr(text, source_lang='auto', target_lang ='ko',max_length = 1000 ):
    '''
    text에 소스 랭기지를 타겟 랭기지로 번역
    sample = apparel_df.review_body.iloc[0]
    translate_kr(sample, source_lang='en', target_lang ='ko',max_length = 1000 )    
    '''    
    if pd.isnull(text):
        return text
    
    tr_text = None
    
    # print("max_length: ", max_length)
    
    text = text[0:max_length] # 1000 자로 한정
    
    try:
        result = translate.translate_text(Text=text, 
                    SourceLanguageCode= source_lang, TargetLanguageCode= target_lang)
        tr_text = result.get('TranslatedText')
    except Exception as ex:
        print("Error occured with {}: ".format(text))
        print(ex)
         
    return tr_text

In [6]:
from functools import partial
# from utils import process_df_parallel, translate_en_df, translate_en

def main_translate(clean_df, num_text, n_cores, saved_file_path, max_length):
    '''
    번역 메인 함수
    '''
    input_clean_df = clean_df.copy()
    input_clean_df = input_clean_df[0:num_text]
    print("clean_df shape: ", input_clean_df.shape)    
    tr_df = process_df_parallel(
                            input_clean_df, 
                            partial(translate_en_df,max_length=max_length),
                            n_cores=n_cores
                            )       

    print("Translation is completed")

    tr_df.to_csv(saved_file_path, index=None)
    print("file: {} is saved".format(saved_file_path))
    
    return tr_df

In [7]:
import os
Category = 'grocery'
num_text = grocery_df.shape[0] 
# num_text = apparel_df.shape[0] 
# num_text = 10 
n_cores = 4

preproc_translate_folder = "preproc_data/translated"
os.makedirs(preproc_translate_folder, exist_ok=True)
    
full_translated_file_name = f"full_translated_{Category}_" + str(num_text) + ".csv"    
full_translated_file_path = os.path.join(preproc_translate_folder, full_translated_file_name)

## 번역 실행 및 결과 저장

In [8]:
%%time
max_length = 1000
full_tr_df = main_translate(grocery_df, num_text, n_cores, full_translated_file_path, max_length)
%store full_translated_file_path

clean_df shape:  (22009, 5)
Translation is completed
file: preproc_data/translated/full_translated_grocery_22009.csv is saved
Stored 'full_translated_file_path' (str)
CPU times: user 619 ms, sys: 191 ms, total: 810 ms
Wall time: 12min 35s


In [9]:
full_translated_file_path

'preproc_data/translated/full_translated_grocery_22009.csv'